In [1]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import json
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
from urllib.request import urlopen 
import io
import getpass
import re
from collections import defaultdict

import nltk
from nltk.collocations import *

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

!pip install sddk
import sddk


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
### CLTK with its data - not needed
!pip install cltk
from cltk.corpus.utils.importer import CorpusImporter
corpus_importer = CorpusImporter('greek')
corpus_importer.import_corpus('greek_models_cltk')
from cltk.stop.greek.stops import STOPS_LIST
from cltk.stem.lemma import LemmaReplacer
lemmatizer = LemmaReplacer('greek')
from cltk.tag.pos import POSTag
tagger = POSTag('greek')
from cltk.tag import ner

In [0]:
### to get an actual version
!pip install anda
from anda.gr import *

In [2]:
conf = sddk.configure_session_and_url("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [0]:
### FROM OUR DIR
morpheus_by_lemma = sddk.read_file("SDAM_data/OGL/morpheus_by_lemma.json", "dict", conf)
morpheus_dict = sddk.read_file("SDAM_data/OGL/morpheus_dict.json", "dict", conf)

In [5]:
### FROM OUR DIR
morpheus_by_lemma = sddk.read_file("https://sciencedata.dk/shared/b4e4546b476b7aef3510c6b7d404b828?download", "dict")
morpheus_dict = sddk.read_file("https://sciencedata.dk/shared/44278699e5ca77abfd06b35729414917?download", "dict")

file import failed
file import failed


In [4]:
morpheus_by_lemma = json.loads(requests.get("https://sciencedata.dk/shared/b4e4546b476b7aef3510c6b7d404b828?download").content)
morpheus_dict = json.loads(requests.get("https://sciencedata.dk/shared/44278699e5ca77abfd06b35729414917?download").content)

UnicodeDecodeError: ignored

# Load MorpheusUnicode.xml and build morpheus_df and morpheus_dict

In [0]:
%%time
response = s.get(sddk_url + "SDAM_data/OGL/MorpheusUnicode.xml")
print(response.text[:200])

<?xml version="1.0"?>
<d>
  <token>
    <word-form-id>1</word-form-id>
    <word-form>ἀάατον</word-form>
    <word-form-without-diacritics>ααατον</word-form-without-diacritics>
    <lemma>ἀάατος</lemm
CPU times: user 861 ms, sys: 243 ms, total: 1.1 s
Wall time: 19.9 s


In [0]:
print(response.text[300000:300500])

a>
  </t>
  <t>
    <i>1631</i>
    <f>ἀγακλέᾰς</f>
    <b>αγακλεᾰς</b>
    <l>ἀγακλεής</l>
    <e>αγακλεης</e>
    <p>a-p---ma-</p>
    <d>76</d>
    <s>very glorious, famous</s>
    <a>doric aeolic</a>
  </t>
  <t>
    <i>1632</i>
    <f>ἀγακλέης</f>
    <b>αγακλεης</b>
    <l>ἀγακλεής</l>
    <e>αγακλεης</e>
    <p>a-p---ma-</p>
    <d>76</d>
    <s>very glorious, famous</s>
    <a>attic epic doric</a>
  </t>
  <t>
    <i>1633</i>
    <f>ἀγακλεής</f>
    <b>αγακλεης</b>
    <l>ἀγακλεής</l>
  


In [0]:
%%time 
### parse the string with the xml file content
root = ET.fromstring(response.text)

CPU times: user 18.2 s, sys: 1.26 s, total: 19.4 s
Wall time: 19.4 s


In [0]:
### look at its root tag
root.tag

'd'

In [0]:
### how many elements we have here
len(root.getchildren())

958477

In [0]:
###  Build morpheus dataframe
%%time
morpheus = []
for token in root.iter("t"):
  i = token.find("i").text # word-form-id
  f = token.find("f").text # word-form
  b = token.find("b").text # word-form-without-diacritics
  l = token.find("l").text # lemma
  e = token.find("e").text # lemma-without-diacritics
  p = token.find("p").text # morphological-analysis
  d = token.find("d").text # lemma-id
  s = token.find("s").text # translation
  a = token.find("a").text # dialect 
  morpheus.append({"i":i, "f":f, "b":b, "l":l, "e":e, "p":p, "d":d, "s":s, "a":a})
morpheus_df = pd.DataFrame(morpheus)

CPU times: user 4.87 s, sys: 173 ms, total: 5.04 s
Wall time: 5.05 s


In [0]:
morpheus_df.uni

,i,f,b,l,e,p,d,s,a
0,1,ἀάατον,ααατον,ἀάατος,ααατος,a-s---fa-,1,"not to be injured, inviolable",None
1,2,ἀάᾱτον,ααᾱτον,ἀάατος,ααατος,a-s---fa-,1,"not to be injured, inviolable",None
2,3,ἀάατος,ααατος,ἀάατος,ααατος,a-s---fn-,1,"not to be injured, inviolable",None
3,4,ἀάᾰτος,ααᾰτος,ἀάατος,ααατος,a-s---fn-,1,"not to be injured, inviolable",None
4,5,ἀάᾱτος,ααᾱτος,ἀάατος,ααατος,a-s---fn-,1,"not to be injured, inviolable",None
...,...,...,...,...,...,...,...,...,...
958471,958472,ὠπός,ωπος,ὤψ,ωψ,n-s---fg-,36488,"the eye, face, countenance",None
958472,958473,ὤψ,ωψ,ὤψ,ωψ,n-s---fn-,36488,"the eye, face, countenance",None
958473,958474,ὦψ,ωψ,ὤψ,ωψ,n-s---fn-,36488,"the eye, face, countenance",None
958474,958475,ὤψ,ωψ,ὤψ,ωψ,n-s---fv-,36488,"the eye, face, countenance",None


In [0]:
###  Build morpheus dictionary (which is much faster)
%%time
morpheus_dict = defaultdict(list)
fs = []
for token in root.iter("t"):
  i = token.find("i").text # word-form-id
  f = token.find("f").text # word-form
  b = token.find("b").text # word-form-without-diacritics
  l = token.find("l").text # lemma
  e = token.find("e").text # lemma-without-diacritics
  p = token.find("p").text # morphological-analysis
  d = token.find("d").text # lemma-id
  s = token.find("s").text # translation
  a = token.find("a").text # dialect 
  token_data = {"i": i, "f":f, "b":b, "l":l, "e":e, "p":p, "d":d, "s":s, "a":a}  
  morpheus_dict[f].append(token_data)
  morpheus_dict[b].append(token_data)

CPU times: user 3.96 s, sys: 0 ns, total: 3.96 s
Wall time: 3.97 s


In [0]:
###  Build reduced morpheus dictionary with unique lemmata as keys
%%time
morpheus_by_lemma = defaultdict(list)
ds = []
for token in root.iter("t"):
  d = int(token.find("d").text) # lemma-id
  l = token.find("l").text # lemma
  s = token.find("s").text # translation
  a = token.find("a").text # dialect 
  token_data = {"d":d, "l":l, "s":s, "a":a}
  if d not in ds:
    morpheus_by_lemma[l].append(token_data)
    ds.append(d)

CPU times: user 3min 2s, sys: 0 ns, total: 3min 2s
Wall time: 3min 2s


In [0]:
morpheus_by_lemma = dict(morpheus_by_lemma)

In [0]:
s.put(sddk_url + "SDAM_data/OGL/morpheus_by_lemma.json", data=json.dumps(morpheus_by_lemma))

<Response [201]>

In [0]:
morpheus_by_lemma["λόγος"]

[{'a': None, 'd': 19869, 'l': 'λόγος', 's': 'the word'}]

In [0]:
morpheus_dict = dict(morpheus_dict)

In [0]:
s.put(sddk_url + "SDAM_data/OGL/morpheus_dict.json", data=json.dumps(morpheus_dict))

<Response [204]>

# Lemmatization and POSTAGs

In [0]:
%%time
morpheus_dict = json.loads(s.get(sddk_url + "SDAM_data/OGL/morpheus_dict.json").content)

CPU times: user 10.6 s, sys: 1.86 s, total: 12.4 s
Wall time: 41.6 s


In [0]:
len(morpheus_dict)

706506

In [0]:
morpheus_by_lemma = json.loads(s.get(sddk_url + "SDAM_data/OGL/morpheus_by_lemma.json").content)

In [0]:
%%time
data_merged_raw = pd.DataFrame(s.get(sddk_url + "SDAM_data/OGL/data_merged_raw.json").json())
data_merged_raw.head(5)

CPU times: user 19.9 s, sys: 7.99 s, total: 27.9 s
Wall time: 1min 12s


In [0]:
data_merged_raw.head(5)

,filename,author,title,string
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...


In [0]:
stopwords_string = " ".join(STOPS_LIST)

In [0]:
stopwords_string

"αὐτὸς αὐτός γε γὰρ γάρ δ' δαὶ δαὶς δαί δαίς διὰ διά δὲ δέ δὴ δή εἰ εἰμὶ εἰμί εἰς εἴμι κατὰ κατά καὶ καί μετὰ μετά μὲν μέν μὴ μή οἱ οὐ οὐδεὶς οὐδείς οὐδὲ οὐδέ οὐκ οὔτε οὕτως οὖν οὗτος παρὰ παρά περὶ περί πρὸς πρός σὸς σός σὺ σὺν σύ σύν τε τι τις τοιοῦτος τοὶ τοί τοὺς τούς τοῦ τὰ τά τὴν τήν τὶ τὶς τί τίς τὸ τὸν τό τόν τῆς τῇ τῶν τῷ ἀλλ' ἀλλὰ ἀλλά ἀπὸ ἀπό ἂν ἄλλος ἄν ἄρα ἐγὼ ἐγώ ἐκ ἐξ ἐμὸς ἐμός ἐν ἐπὶ ἐπί ἐὰν ἐάν ἑαυτοῦ ἔτι ἡ ἢ ἤ ὁ ὃδε ὃς ὅδε ὅς ὅστις ὅτι ὑμὸς ὑμός ὑπὲρ ὑπέρ ὑπὸ ὑπό ὡς ὥστε ὦ ξύν ξὺν σύν σὺν τοῖς τᾶς την α μην ἃ 𝔚 β δη δι δ᾿ δʼ δ τότ ἀλλʼ ὅσʼ ἐπʼ ιη △ζ ιβ τχ μη ; ὃ γ . ὅταν ποτέ οὐδʼ καθʼ ἀλλ᾿ την α μην ἃ 𝔚 β δη δι δ᾿ δʼ δ τότ ἀλλʼ ὅσʼ ἐπʼ ιη △ζ ιβ τχ μη ; ὃ γ ὅταν ποτέ οὐδʼ καθʼ ἀλλ᾿ την α μην ἃ 𝔚 β δη δι δ᾿ δʼ δ τότ ἀλλʼ ὅσʼ ἐπʼ ιη △ζ ιβ τχ μη ὃ γ ὅταν ποτέ οὐδʼ καθʼ ἀλλ᾿"

In [0]:
stopwords_string.encode("utf-8")

b"\xce\xb1\xe1\xbd\x90\xcf\x84\xe1\xbd\xb8\xcf\x82 \xce\xb1\xe1\xbd\x90\xcf\x84\xcf\x8c\xcf\x82 \xce\xb3\xce\xb5 \xce\xb3\xe1\xbd\xb0\xcf\x81 \xce\xb3\xce\xac\xcf\x81 \xce\xb4' \xce\xb4\xce\xb1\xe1\xbd\xb6 \xce\xb4\xce\xb1\xe1\xbd\xb6\xcf\x82 \xce\xb4\xce\xb1\xce\xaf \xce\xb4\xce\xb1\xce\xaf\xcf\x82 \xce\xb4\xce\xb9\xe1\xbd\xb0 \xce\xb4\xce\xb9\xce\xac \xce\xb4\xe1\xbd\xb2 \xce\xb4\xce\xad \xce\xb4\xe1\xbd\xb4 \xce\xb4\xce\xae \xce\xb5\xe1\xbc\xb0 \xce\xb5\xe1\xbc\xb0\xce\xbc\xe1\xbd\xb6 \xce\xb5\xe1\xbc\xb0\xce\xbc\xce\xaf \xce\xb5\xe1\xbc\xb0\xcf\x82 \xce\xb5\xe1\xbc\xb4\xce\xbc\xce\xb9 \xce\xba\xce\xb1\xcf\x84\xe1\xbd\xb0 \xce\xba\xce\xb1\xcf\x84\xce\xac \xce\xba\xce\xb1\xe1\xbd\xb6 \xce\xba\xce\xb1\xce\xaf \xce\xbc\xce\xb5\xcf\x84\xe1\xbd\xb0 \xce\xbc\xce\xb5\xcf\x84\xce\xac \xce\xbc\xe1\xbd\xb2\xce\xbd \xce\xbc\xce\xad\xce\xbd \xce\xbc\xe1\xbd\xb4 \xce\xbc\xce\xae \xce\xbf\xe1\xbc\xb1 \xce\xbf\xe1\xbd\x90 \xce\xbf\xe1\xbd\x90\xce\xb4\xce\xb5\xe1\xbd\xb6\xcf\x82 \xce\xbf\xe1\xbd\x9

In [0]:
s.put("https://sciencedata.dk/files/anda_data/testfile.txt", data=stopwords_string.encode("utf-8"))

UnicodeEncodeError: ignored

In [0]:
print

In [0]:
ast.literal_eval(requests.get("https://sciencedata.dk/shared/fae9557f10bdca2ae2b06c1839754324?download").text).encode("utf-8")

SyntaxError: ignored

In [0]:
STOPS_LIST = .split()

['αὐτὸς',
 'αὐτός',
 'γε',
 'γὰρ',
 'γάρ',
 "δ'",
 'δαὶ',
 'δαὶς',
 'δαί',
 'δαίς',
 'διὰ',
 'διά',
 'δὲ',
 'δέ',
 'δὴ',
 'δή',
 'εἰ',
 'εἰμὶ',
 'εἰμί',
 'εἰς',
 'εἴμι',
 'κατὰ',
 'κατά',
 'καὶ',
 'καί',
 'μετὰ',
 'μετά',
 'μὲν',
 'μέν',
 'μὴ',
 'μή',
 'οἱ',
 'οὐ',
 'οὐδεὶς',
 'οὐδείς',
 'οὐδὲ',
 'οὐδέ',
 'οὐκ',
 'οὔτε',
 'οὕτως',
 'οὖν',
 'οὗτος',
 'παρὰ',
 'παρά',
 'περὶ',
 'περί',
 'πρὸς',
 'πρός',
 'σὸς',
 'σός',
 'σὺ',
 'σὺν',
 'σύ',
 'σύν',
 'τε',
 'τι',
 'τις',
 'τοιοῦτος',
 'τοὶ',
 'τοί',
 'τοὺς',
 'τούς',
 'τοῦ',
 'τὰ',
 'τά',
 'τὴν',
 'τήν',
 'τὶ',
 'τὶς',
 'τί',
 'τίς',
 'τὸ',
 'τὸν',
 'τό',
 'τόν',
 'τῆς',
 'τῇ',
 'τῶν',
 'τῷ',
 "ἀλλ'",
 'ἀλλὰ',
 'ἀλλά',
 'ἀπὸ',
 'ἀπό',
 'ἂν',
 'ἄλλος',
 'ἄν',
 'ἄρα',
 'ἐγὼ',
 'ἐγώ',
 'ἐκ',
 'ἐξ',
 'ἐμὸς',
 'ἐμός',
 'ἐν',
 'ἐπὶ',
 'ἐπί',
 'ἐὰν',
 'ἐάν',
 'ἑαυτοῦ',
 'ἔτι',
 'ἡ',
 'ἢ',
 'ἤ',
 'ὁ',
 'ὃδε',
 'ὃς',
 'ὅδε',
 'ὅς',
 'ὅστις',
 'ὅτι',
 'ὑμὸς',
 'ὑμός',
 'ὑπὲρ',
 'ὑπέρ',
 'ὑπὸ',
 'ὑπό',
 'ὡς',
 'ὥστε',
 'ὦ',
 'ξύν',
 'ξὺν',
 'σύν',
 

In [0]:
### simple replacements
to_replace_dict={
    "ἓ":"ἕ",
    "ὸ" : "ό",
    "ὰ" : "ά",
    "ὲ" : "έ",
    "ὶ" : "ί",
    "ὴ" : "η",
    }

STOPS_LIST = [unicodedata.normalize("NFC", stopword) for stopword in STOPS_LIST]
STOPS_LIST += ['την', 'α', 'μην', 'ἃ', '𝔚','β', 'δη','δι', 'δ᾿','δʼ', 'δ', 'τότ', 'ἀλλʼ', 'ὅσʼ', 'ἐπʼ', 'ιη','△ζ', 'ιβ', 'τχ', 'μη', 'ὃ', 'γ', 'ὅταν', 'ποτέ', 'οὐδʼ', 'καθʼ', 'ἀλλ᾿'] ### here you can add any stop words you want to omit from the analyses

def get_sentences(string):
  sentences = [s.strip() for s in re.split("\·|\.|\:|\;", unicodedata.normalize("NFC", string))]
  return sentences

def return_list_of_tokens(word, filter_by_postag=None, involve_unknown=False):
  word = unicodedata.normalize("NFC", word)
  try:
    list_of_tokens = morpheus_dict[word]
    if len(list_of_tokens) < 1:
      list_of_tokens = morpheus_dict[word.lower()]
      if len(list_of_tokens) < 1:
        list_of_tokens = [{"f":word, "i": "", "b":"", "l":word.lower(), "e":"", "p":"", "d":"", "s":"", "a":""}]
  except:
    list_of_tokens = [{"f":word, "i": "", "b":"", "l":word.lower(), "e":"", "p":"", "d":"", "s":"", "a":""}]

  if filter_by_postag != None:
    try:
      list_of_tokens_filtered = []
      for token in list_of_tokens:
        if token["p"][0] in filter_by_postag:
          list_of_tokens_filtered.append(token)
      list_of_tokens = list_of_tokens_filtered
    except:
      if involve_unknown == False:
        list_of_tokens = []
  return list_of_tokens

def return_all_unique_lemmata(word, filter_by_postag=None, involve_unknown=False):
  list_of_tokens = return_list_of_tokens(word, filter_by_postag=filter_by_postag, involve_unknown=involve_unknown)
  lemmata = "/".join(set([token["l"] for token in list_of_tokens]))
  return lemmata

def return_all_unique_translations(word, filter_by_postag=None, involve_unknown=False):
  list_of_tokens = return_list_of_tokens(word, filter_by_postag=filter_by_postag, involve_unknown=involve_unknown)
  try:
    translations = " / ".join(set([token["s"] for token in list_of_tokens]))
  except:
    translations = ""
  return translations

def return_first_lemma(word, filter_by_postag=None, involve_unknown=False):
  list_of_tokens = return_list_of_tokens(word, filter_by_postag=filter_by_postag, involve_unknown=involve_unknown)
  try:
    first_lemma = list_of_tokens[0]["l"]
  except:
    first_lemma = ""
  return first_lemma

def morphological_analysis(string):
  string_tokenized = tokenize_string(string)
  string_analyzed = [return_list_of_tokens(word)[0] for word in string_tokenized if word != ""]
  for element in string_analyzed:
    try: 
      first_tokens.append(element[0])
    except:
      pass
  return first_tokens

def lemma_translator(word):
  try:
    translations = []
    for option in word.split("/"):
      translations.append(" / ".join(set([token["s"] for token in morpheus_by_lemma[option]])))
    translations = " / ".join(translations)
  except:
    translations = ""
  return translations

def tokenize_string(string):
  string = re.sub(r'[A-Za-z0-9]+', "", string)
  string = re.sub(r'[-,\(\)=\\\?·‖\+;\.\:/\[\]\*—»«\§˘„”\|]+', "", string)
  string = re.sub(r'[^\w\s]','', string)
  for k,v in to_replace_dict.items():
    string = string.replace(k,v)
  string = unicodedata.normalize("NFC", string)
  string_tokenized = string.split()
  string_tokenized = [word for word in string_tokenized if len(word) > 1]
  return string_tokenized

def lemmatize_string(string, all_lemmata=False, filter_by_postag=None, involve_unknown=False):
  string_tokenized = tokenize_string(string)
  string_tokenized = [word for word in string_tokenized if word not in STOPS_LIST]
  if all_lemmata==True:
    string_lemmatized = [return_all_unique_lemmata(word, filter_by_postag=filter_by_postag, involve_unknown=involve_unknown) for word in string_tokenized if word != ""]
  else: 
    string_lemmatized = [return_first_lemma(word, filter_by_postag=filter_by_postag, involve_unknown=involve_unknown) for word in string_tokenized if word != ""]  
  string_lemmatized = [word for word in string_lemmatized if word != ""]
  string_lemmatized = [re.sub(r'\d', "", w) for w in string_lemmatized if w not in STOPS_LIST]
  return string_lemmatized

def get_lemmatized_sentences(string, all_lemmata=False, filter_by_postag=None, involve_unknown=False):
  sentences = get_sentences(string)
  lemmatized_sentences = []
  for sentence in sentences:
    lemmatized_sentence = lemmatize_string(sentence, all_lemmata=all_lemmata, filter_by_postag=filter_by_postag, involve_unknown=involve_unknown)
    if len(lemmatized_sentence) > 0:
      lemmatized_sentences.append(lemmatized_sentence)
  return lemmatized_sentences

# Get Lemmatized Sentences for the Whole Corpus

In [0]:
%%time
data_merged_raw["lem_senteces"] = data_merged_raw.apply(lambda row: get_lemmatized_sentences(row["string"], all_lemmata=True, filter_by_postag=["n","a","v"]), axis=1)

CPU times: user 8min 30s, sys: 628 ms, total: 8min 30s
Wall time: 8min 31s


In [0]:
data_merged_raw.head(5)

,filename,author,title,string,lem_senteces
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...,"[[ἀνώνυμος, ἀναμέτρησις, οἰκουμένη, εἶμι/ἀίω/ἤ..."
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...,"[[μάταιος], [πηρός/Πάρος, ἐμέω/ἐμός], [ἵημι], ..."
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...","[[πινυτός, ἀντίγραφος/ἀντιγραφή, θαυμάζω, ἀποδ..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...,"[[κόσμος/κοσμέω, ἀλέξανδρος], [δαιμόνιον/δαιμο..."
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...,"[[πνεῦμα], [ἀτίζω/τίνω, ἔμφυτος, πνεῦμα, αὔξησ..."


In [0]:
s.put(sddk_url + "SDAM_data/OGL/data_merged_lem.json", data=data_merged_raw.to_json())

<Response [201]>

# Testing lemmatization and translation

In [0]:
[(word, return_all_unique_translations(word)) for word in tokenize_string(test_string)]

[('ΑΝΑΛΥΤΙΚΩΝ', ''),
 ('ΠΡΟΤΕΡΩΝ',
  'prae. / to be before, be in advance / to speak to, address, accost'),
 ('ΠΡΩΤΟΝ', 'prae.'),
 ('εἰπεῖν', 'to speak, say'),
 ('τίνος', 'any one, any thing, some one, some thing; / who? which?'),
 ('ἐστίν', 'I have'),
 ('σκέψις', 'a viewing, perception by the senses'),
 ('ἀπόδειξιν', 'a shewing forth, exhibiting'),
 ('ἐπιστήμης', 'acquaintance with'),
 ('ἀποδεικτικῆς', ''),
 ('εἶτα', 'then, next'),
 ('διορίσαι', 'to draw a boundary through, divide by limits, separate'),
 ('ἐστι', 'I have'),
 ('πρότασις', 'a proposition, the premiss'),
 ('ὅρος', 'a boundary, landmark'),
 ('συλλογισμός', 'computation'),
 ('ποῖος', 'of a certain nature, kind / of what nature? of what sort?'),
 ('τέλειος', 'having reached its end, finished, complete'),
 ('ποῖος', 'of a certain nature, kind / of what nature? of what sort?'),
 ('ἀτελής', 'without end'),
 ('ταῦτα', 'this'),
 ('ὅλῳ', ''),
 ('εἶναι', 'I have'),
 ('εἶναι', 'I have'),
 ('τόδε', 'this'),
 ('τῷδε', 'this'),
 ('λέγ

In [0]:
test_string = data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist()[0]
print(test_string)

ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. ΠΡΩΤΟΝ εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις, ὅτι περὶ ἀπόδειξιν καὶ ἐπιστήμης ἀποδεικτικῆς· εἶτα διορίσαι τί ἐστι πρότασις καὶ τί ὅρος καὶ τί συλλογισμός, καὶ ποῖος τέλειος καὶ ποῖος ἀτελής, μετὰ δὲ ταῦτα τί τὸ ἐν ὅλῳ εἶναι ἢ μὴ εἶναι τόδε τῷδε, καὶ τί λέγομεν τὸ κατὰ παντὸς ἢ μηδενὸς κατηγορεῖσθαι. Πρότασις μὲν οὖν ἐστὶ λόγος καταφατικὸς ἢ ἀποφατικὸς τινὸς κατά τινος. Οὗτος δὲ ἢ καθόλου ἢ ἐν μέρει ἢ ἀδιόριστος. Λέγω δὲ καθόλου μὲν τὸ παντὶ ἢ μηδενὶ ὑπάρχειν, ἐν μέρει δὲ τὸ τινὶ ἢ μὴ τινὶ ἢ μὴ παντὶ ὑπάρχειν, ἀδιόριστον δὲ τὸ ὑπάρχειν ἢ μὴ ὑπάρχειν ἄνευ τοῦ καθόλου, ἢ κατὰ μέρος, οἷον τὸ τῶν ἐναντίων εἶναι τὴν αὐτὴν ἐπιστήμην ἢ τὸ τὴν ἡδονὴν μὴ εἶναι ἀγαθόν. Διαφέρει δὲ ἡ ἀποδεικτικὴ πρότασις τῆς διαλεκτικῆς, ὅτι ἡ μὲν ἀποδεικτικὴ λῆψις θατέρου μορίου τῆς ἀντιφάσεώς ἐστιν (οὐ γὰρ ἐρωτᾷ ἀλλὰ λαμβάνει ὁ ἁποδεικνύων), ἡ δὲ διαλεκτικὴ ἐρώτησις ἀντιφάσεώς ἐστιν. Οὐδὲν δὲ διοίσει πρὸς τὸ γενέσθαι τὸν ἑκατέρου συλλογισμόν· καὶ γὰρ ὁ ἀποδεικνύων καὶ ὁ ἐρωτῶν συλλογίζε

In [0]:
get_lemmatized_sentences(test_string, all_lemmata=True, filter_by_postag=["n","a","v"])

[['πρότερος'],
 ['πρότερος', 'εἶπον', 'σκέψις', 'ἀπόδειξις', 'ἐπιστήμη'],
 ['διορίζω',
  'πρότασις',
  'ὅρος',
  'συλλογισμός',
  'ποῖος/ποιός',
  'τέλειος',
  'ποῖος/ποιός',
  'ἀτελής',
  'ὅλοξ/ὁλάω',
  'λέγω',
  'πᾶς',
  'μηδείς',
  'κατηγορέω'],
 ['πρότασις', 'λόγος'],
 ['μέρος'],
 ['λέγω',
  'πᾶς',
  'μηδείς',
  'ὑπάρχω',
  'μέρος',
  'πᾶς',
  'ὑπάρχω',
  'ὑπάρχω',
  'ὑπάρχω',
  'ἀνίημι/ἄνω',
  'μέρος',
  'οἷος',
  'ἐναντίος',
  'ἐπιστήμη',
  'ἀγαθός'],
 ['διαφέρω',
  'πρότασις',
  'διαλεκτικός',
  'λῆψις',
  'ἕτερος',
  'μόριος/μόριον',
  'ἐρωτάω',
  'λαμβάνω',
  'διαλεκτικός',
  'ἐρώτησις'],
 ['διαφέρω', 'γίγνομαι', 'ἑκάτερος', 'συλλογισμός'],
 ['ἔρως/ἐρωτάω', 'συλλογίζομαι', 'λαβή', 'ὑπάρχω', 'ὑπάρχω'],
 ['πρότασις',
  'ἁπλόω',
  'τρόπος/τροπός',
  'ἀρχή',
  'ὑπόθεσις',
  'διαλεκτικός',
  'ἐρώτησις',
  'λῆψις',
  'ἔνδοξος',
  'τοπικός',
  'ἔρομαι/ἐρῶ/εἴρω'],
 ['πρότασις',
  'διαφέρω',
  'διαλεκτικός',
  'ἀκρίβεια',
  'ἐρῶ',
  'χρεία',
  'ἱκανόω/ἱκανός',
  'ὁρίζω/ὀρίνω'],
 ['ὀρός

In [0]:
print(lemmatize_string(test_string, all_lemmata=True, filter_by_postag=["n","a","v"], involve_unknown=True)[:100])

['αναλυτικων', 'πρότερος', 'πρότερος', 'εἶπον', 'σκέψις', 'ἀπόδειξις', 'ἐπιστήμη', 'ἀποδεικτικῆς', 'διορίζω', 'πρότασις', 'ὅρος', 'συλλογισμός', 'ποῖος/ποιός', 'τέλειος', 'ποῖος/ποιός', 'ἀτελής', 'ὅλοξ/ὁλάω', 'λέγω', 'πᾶς', 'μηδείς', 'κατηγορέω', 'πρότασις', 'λόγος', 'καταφατικός', 'ἀποφατικός', 'μέρος', 'ἀδιόριστος', 'λέγω', 'πᾶς', 'μηδείς', 'ὑπάρχω', 'μέρος', 'πᾶς', 'ὑπάρχω', 'ἀδιόριστον', 'ὑπάρχω', 'ὑπάρχω', 'ἀνίημι/ἄνω', 'μέρος', 'οἷος', 'ἐναντίος', 'αὐτην', 'ἐπιστήμη', 'ἡδονην', 'ἀγαθός', 'διαφέρω', 'ἀποδεικτικη', 'πρότασις', 'διαλεκτικός', 'ἀποδεικτικη', 'λῆψις', 'ἕτερος', 'μόριος/μόριον', 'ἀντιφάσεώς', 'ἐρωτάω', 'λαμβάνω', 'ἁποδεικνύων', 'διαλεκτικός', 'ἐρώτησις', 'ἀντιφάσεώς', 'διαφέρω', 'γίγνομαι', 'ἑκάτερος', 'συλλογισμός', 'ἔρως/ἐρωτάω', 'συλλογίζομαι', 'λαβή', 'ὑπάρχω', 'ὑπάρχω', 'συλλογιστικη', 'πρότασις', 'ἁπλόω', 'κατάφασις', 'ἀπόφασίς', 'τρόπος/τροπός', 'ἀποδεικτικη', 'ἀληθης', 'ἀρχή', 'ὑπόθεσις', 'εἰλημμένη', 'διαλεκτικός', 'ἐρώτησις', 'ἀντιφάσεως', 'συλλογιζομένῳ', 'λ

In [0]:
word = "ἀποδεικτικῆς"
return_list_of_tokens(word)

[{'a': '',
  'b': '',
  'd': '',
  'e': '',
  'f': 'ἀποδεικτικῆς',
  'i': '',
  'l': 'ἀποδεικτικῆς',
  'p': '',
  's': ''}]

In [0]:
sentences = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist()[:5]:
  sentences.extend(get_sentences(string))
print(sentences)

['ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α', 'ΠΡΩΤΟΝ εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις, ὅτι περὶ ἀπόδειξιν καὶ ἐπιστήμης ἀποδεικτικῆς', 'εἶτα διορίσαι τί ἐστι πρότασις καὶ τί ὅρος καὶ τί συλλογισμός, καὶ ποῖος τέλειος καὶ ποῖος ἀτελής, μετὰ δὲ ταῦτα τί τὸ ἐν ὅλῳ εἶναι ἢ μὴ εἶναι τόδε τῷδε, καὶ τί λέγομεν τὸ κατὰ παντὸς ἢ μηδενὸς κατηγορεῖσθαι', 'Πρότασις μὲν οὖν ἐστὶ λόγος καταφατικὸς ἢ ἀποφατικὸς τινὸς κατά τινος', 'Οὗτος δὲ ἢ καθόλου ἢ ἐν μέρει ἢ ἀδιόριστος', 'Λέγω δὲ καθόλου μὲν τὸ παντὶ ἢ μηδενὶ ὑπάρχειν, ἐν μέρει δὲ τὸ τινὶ ἢ μὴ τινὶ ἢ μὴ παντὶ ὑπάρχειν, ἀδιόριστον δὲ τὸ ὑπάρχειν ἢ μὴ ὑπάρχειν ἄνευ τοῦ καθόλου, ἢ κατὰ μέρος, οἷον τὸ τῶν ἐναντίων εἶναι τὴν αὐτὴν ἐπιστήμην ἢ τὸ τὴν ἡδονὴν μὴ εἶναι ἀγαθόν', 'Διαφέρει δὲ ἡ ἀποδεικτικὴ πρότασις τῆς διαλεκτικῆς, ὅτι ἡ μὲν ἀποδεικτικὴ λῆψις θατέρου μορίου τῆς ἀντιφάσεώς ἐστιν (οὐ γὰρ ἐρωτᾷ ἀλλὰ λαμβάνει ὁ ἁποδεικνύων), ἡ δὲ διαλεκτικὴ ἐρώτησις ἀντιφάσεώς ἐστιν', 'Οὐδὲν δὲ διοίσει πρὸς τὸ γενέσθαι τὸν ἑκατέρου συλλογισμόν', 'καὶ γὰρ ὁ ἀποδεικνύων καὶ 

In [0]:
sentences = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist()[:5]:
  sentences.extend(get_lemmatized_sentences(string, all_lemmata=True, filter_by_postag=["n", "a", "v"]))

In [0]:
print(sentences[:10])

[['πρότερος'], ['πρότερος', 'εἶπον', 'σκέψις', 'ἀπόδειξις', 'ἐπιστήμη'], ['διορίζω', 'πρότασις', 'ὅρος', 'συλλογισμός', 'ποῖος/ποιός', 'τέλειος', 'ποῖος/ποιός', 'ἀτελής', 'ὅλοξ/ὁλάω', 'λέγω', 'πᾶς', 'μηδείς', 'κατηγορέω'], ['πρότασις', 'λόγος'], ['μέρος'], ['λέγω', 'πᾶς', 'μηδείς', 'ὑπάρχω', 'μέρος', 'πᾶς', 'ὑπάρχω', 'ὑπάρχω', 'ὑπάρχω', 'ἀνίημι/ἄνω', 'μέρος', 'οἷος', 'ἐναντίος', 'ἐπιστήμη', 'ἀγαθός'], ['διαφέρω', 'πρότασις', 'διαλεκτικός', 'λῆψις', 'ἕτερος', 'μόριος/μόριον', 'ἐρωτάω', 'λαμβάνω', 'διαλεκτικός', 'ἐρώτησις'], ['διαφέρω', 'γίγνομαι', 'ἑκάτερος', 'συλλογισμός'], ['ἔρως/ἐρωτάω', 'συλλογίζομαι', 'λαβή', 'ὑπάρχω', 'ὑπάρχω'], ['πρότασις', 'ἁπλόω', 'τρόπος/τροπός', 'ἀρχή', 'ὑπόθεσις', 'διαλεκτικός', 'ἐρώτησις', 'λῆψις', 'ἔνδοξος', 'τοπικός', 'ἔρομαι/ἐρῶ/εἴρω']]


In [0]:
string_tokenized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_tokenized += tokenize_string(string)

In [0]:
string_tokenized[:100]

['ΑΝΑΛΥΤΙΚΩΝ',
 'ΠΡΟΤΕΡΩΝ',
 'ΠΡΩΤΟΝ',
 'εἰπεῖν',
 'περί',
 'τί',
 'καί',
 'τίνος',
 'ἐστίν',
 'σκέψις',
 'ὅτι',
 'περί',
 'ἀπόδειξιν',
 'καί',
 'ἐπιστήμης',
 'ἀποδεικτικῆς',
 'εἶτα',
 'διορίσαι',
 'τί',
 'ἐστι',
 'πρότασις',
 'καί',
 'τί',
 'ὅρος',
 'καί',
 'τί',
 'συλλογισμός',
 'καί',
 'ποῖος',
 'τέλειος',
 'καί',
 'ποῖος',
 'ἀτελής',
 'μετά',
 'δέ',
 'ταῦτα',
 'τί',
 'τό',
 'ἐν',
 'ὅλῳ',
 'εἶναι',
 'μη',
 'εἶναι',
 'τόδε',
 'τῷδε',
 'καί',
 'τί',
 'λέγομεν',
 'τό',
 'κατά',
 'παντός',
 'μηδενός',
 'κατηγορεῖσθαι',
 'Πρότασις',
 'μέν',
 'οὖν',
 'ἐστί',
 'λόγος',
 'καταφατικός',
 'ἀποφατικός',
 'τινός',
 'κατά',
 'τινος',
 'Οὗτος',
 'δέ',
 'καθόλου',
 'ἐν',
 'μέρει',
 'ἀδιόριστος',
 'Λέγω',
 'δέ',
 'καθόλου',
 'μέν',
 'τό',
 'παντί',
 'μηδενί',
 'ὑπάρχειν',
 'ἐν',
 'μέρει',
 'δέ',
 'τό',
 'τινί',
 'μη',
 'τινί',
 'μη',
 'παντί',
 'ὑπάρχειν',
 'ἀδιόριστον',
 'δέ',
 'τό',
 'ὑπάρχειν',
 'μη',
 'ὑπάρχειν',
 'ἄνευ',
 'τοῦ',
 'καθόλου',
 'κατά',
 'μέρος',
 'οἷον',
 'τό']

In [0]:
morpheus_dict["αἴτιος"]

[{'a': None,
  'b': 'αιτιος',
  'd': '950',
  'e': 'αιτιος',
  'f': 'αἴτιος',
  'i': '28485',
  'l': 'αἴτιος',
  'p': 'a-s---fn-',
  's': 'to blame, blameworthy, culpable'},
 {'a': None,
  'b': 'αιτιος',
  'd': '950',
  'e': 'αιτιος',
  'f': 'αἴτιος',
  'i': '28507',
  'l': 'αἴτιος',
  'p': 'a-s---mn-',
  's': 'to blame, blameworthy, culpable'}]

In [0]:
aristotle_lemmatized = []
all_string_tokenized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_tokenized = tokenize_string(string)
  string_tokenized = [word for word in string_tokenized if word not in STOPS_LIST]
  all_string_tokenized += string_tokenized

  #aristotle_lemmatized += lemmatizer.lemmatize(string_tokenized)

In [0]:
from cltk.tag.pos import POSTag
tagger = POSTag('greek')

In [0]:
postagged = tagger.tag_ngram_123_backoff(" ".join(all_string_tokenized))

In [0]:
len(postagged)

536816

In [0]:
### proportion of postagged words
len([tup for tup in postagged if tup[1] != None]) / len(postagged)

0.5629247265357218

In [0]:
len(postagged)

43

In [0]:
nltk.FreqDist(aristotle_lemmatized).most_common(100)

[('εἰμί', 32232),
 ('οὗτος', 13822),
 ('ὁ', 9785),
 ('αὐτός', 8172),
 ('ἔχω', 7257),
 ('πᾶς', 6549),
 ('γίγνομαι', 6093),
 ('ἄλλος', 4940),
 ('οἷος', 4824),
 ('λέγω1', 4664),
 ('ὑπάρχω', 4301),
 ('τις', 4016),
 ('πρότερος', 4007),
 ('ἵημι', 3508),
 ('ποιέω', 3201),
 ('οὐδείς', 3159),
 ('ὅς', 3095),
 ('τοιοῦτος', 2914),
 ('κινέω', 2892),
 ('ἕτερος', 2872),
 ('ὥσπερ', 2861),
 ('δέω1', 2739),
 ('Β', 2579),
 ('ἀνάγκη', 2449),
 ('συμβαίνω', 2389),
 ('οὐ', 2377),
 ('ὅσος', 2305),
 ('ἐρῶ', 2213),
 ('Α', 2170),
 ('ἕκαστος', 2154),
 ('ὁμοιόω', 2127),
 ('Γ', 2111),
 ('λόγος', 2090),
 ('φύω', 1976),
 ('μᾶλλον', 1967),
 ('ἐναντίος', 1954),
 ('ἐνδέχομαι', 1942),
 ('δοκέω', 1902),
 ('πολύς', 1890),
 ('ἑαυτοῦ', 1848),
 ('λαμβάνω', 1843),
 ('ἄνθρωπος', 1822),
 ('ὅλοξ', 1780),
 ('εἷς', 1758),
 ('ἀγαθός', 1685),
 ('μέγας', 1586),
 ('φημί', 1585),
 ('δῆλος', 1565),
 ('μόνος', 1559),
 ('φαίνω', 1555),
 ('ἀρχή', 1533),
 ('ἐπεί', 1524),
 ('γένος', 1513),
 ('ἀεί', 1491),
 ('μηδείς', 1460),
 ('διό', 1

In [0]:
### Aristotle
string_lemmatized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_lemmatized += lemmatize_string(string, all_lemmata=True, filter_by_postag=["n","a","v"])
nltk.FreqDist(string_lemmatized).most_common(200)

[('πᾶς', 5366),
 ('γίγνομαι', 4870),
 ('οἷος', 4823),
 ('λέγω', 4048),
 ('πρότερος', 3969),
 ('ὑπάρχω', 3959),
 ('ἔχω', 3201),
 ('ἕτερος', 2872),
 ('ἵημι', 2630),
 ('λόγος', 2607),
 ('χέω/ἔχις/ἔχω', 2602),
 ('ποιέω', 2581),
 ('δεῖ/δέω', 2454),
 ('ὅσος', 2306),
 ('ὁμοιόω/ὅμοιος', 2168),
 ('ἕκαστος', 2154),
 ('εἰμί/εἶμι', 2016),
 ('μαλλός', 1969),
 ('πολύς', 1843),
 ('ἐναντίος', 1840),
 ('ἄνθρωπος', 1820),
 ('κινέω', 1814),
 ('ἀνάγκη', 1794),
 ('σῶμα', 1681),
 ('συμβαίνω', 1680),
 ('ἀγαθός', 1675),
 ('ἐνδέχομαι', 1647),
 ('μέγας', 1588),
 ('γένος', 1514),
 ('Δῆλος/δῆλος', 1508),
 ('ἔπειμι/ἔπος', 1462),
 ('μηδείς', 1460),
 ('δοκέω', 1452),
 ('φύσις', 1448),
 ('μέρος', 1407),
 ('φανερός', 1365),
 ('λαμβάνω', 1359),
 ('μόνος', 1330),
 ('γῆ', 1316),
 ('ὕδωρ', 1315),
 ('ὅλοξ', 1312),
 ('χρόνος', 1297),
 ('τρόπος/τροπός', 1280),
 ('συλλογισμός', 1267),
 ('ἀναγκαῖον/ἀναγκαῖος', 1234),
 ('εἰμί/ἐάω', 1185),
 ('φαίνω', 1157),
 ('ἀδύνατος', 1152),
 ('οὐσία', 1150),
 ('τόπος', 1150),
 ('δύναμις', 11

In [0]:
string_tokenized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_tokenized += tokenize_string(string)
nltk.FreqDist(string_tokenized).most_common(200)

[('καί', 51026),
 ('τό', 33541),
 ('δέ', 27198),
 ('γάρ', 18709),
 ('μέν', 15960),
 ('τῶν', 14956),
 ('τά', 10900),
 ('ἢ', 10870),
 ('τοῦ', 10655),
 ('ἐν', 10107),
 ('ἡ', 10037),
 ('την', 9299),
 ('τῷ', 9167),
 ('δʼ', 7810),
 ('εἶναι', 7540),
 ('μη', 6970),
 ('ὁ', 6718),
 ('τῆς', 6650),
 ('τοῖς', 6022),
 ('ὅτι', 5731),
 ('δ', 5683),
 ('εἰ', 5608),
 ('πρός', 5458),
 ('οὐ', 5310),
 ('διά', 5257),
 ('περί', 5209),
 ('τόν', 4911),
 ('οὖν', 4715),
 ('οὐκ', 4573),
 ('οἷον', 4459),
 ('κατά', 4419),
 ('οἱ', 4252),
 ('ἂν', 4068),
 ('τε', 3786),
 ('ἐπί', 3705),
 ('ὡς', 3699),
 ('τοῦτο', 3643),
 ('τι', 3557),
 ('ἐκ', 3464),
 ('ἀλλά', 3326),
 ('εἰς', 3225),
 ('τάς', 3037),
 ('ὥσπερ', 2792),
 ('ἐστιν', 2739),
 ('τοὺς', 2681),
 ('τῇ', 2647),
 ('ἔσται', 2628),
 ('αἱ', 2605),
 ('Β', 2579),
 ('τούτων', 2528),
 ('ἔχει', 2479),
 ('ἐξ', 2448),
 ('ἀλλʼ', 2395),
 ('ἐστίν', 2348),
 ('ἔστι', 2185),
 ('Α', 2170),
 ('Γ', 2111),
 ('δεῖ', 2099),
 ('τις', 2040),
 ('ὅταν', 1992),
 ('ταῦτα', 1972),
 ('μᾶλλον', 1956)

In [0]:
lemma_translator("διά")

'through c. gen. through, by means of c. acc.'

In [0]:
arist_most_common = [tup[0] for tup in nltk.FreqDist(string_lemmatized).most_common(1000)]


In [0]:
morpheus_by_lemma["κοινών"]

[{'a': None, 'd': 18272, 'l': 'κοινών', 's': None}]

In [0]:
lemma_translator("κοινών")

''

In [0]:
for word in arist_most_common:
  print([word, lemma_translator(word)])

['πᾶς', 'all, the whole']
['οἷος', 'such as, what sort']
['γίγνομαι', 'gi-geno']
['λέγω', 'to lay asleep, lull to sleep / to say, speak / to gather, pick up']
['πρότερος', 'prae.']
['ὑπάρχω', 'to begin, make a beginning']
['ἵημι', 'to set a going, put in motion']
['ἔχω', 'to have']
['ἕτερος', 'one of the']
['χέω/ἔχις/ἔχω', 'to pour / an adder, viper / to have']
['λόγος', 'the word']
['ποιέω', 'to make']
['δεῖ/δέω', 'to bind / to bind, tie, fetter / to lack, miss, stand in need of']
['ὅσος', 'quantus']
['ὁμοιόω/ὅμοιος', 'to make like / like, resembling']
['εἰμί/ἐάω/ἀάω', 'I have / to let, suffer, allow, permit / to hurt, damage']
['ἕκαστος', 'every, every one, each, each one']
['εἰμί/εἶμι', 'I have / to go']
['μαλλός', 'a lock of wool, wool']
['πολύς', 'many']
['κινέω', 'to set in motion, to move']
['ἀνάγκη', 'force, constraint, necessity']
['ἐναντίος', 'opposite']
['ἄνθρωπος', 'manfaced']
['σῶμα', 'the body']
['ἀγαθός', 'good']
['ἐνδέχομαι', 'to take upon oneself']
['συμβαίνω', 'to sta

In [0]:

for word in arist_most_common:
  for option in word.split("/"):
    for token in morpheus_by_lemma[option]:
      ([word, option, token["s"]])

In [0]:
"εἰμί/ἐάω/ἵημι/ἑός".split("/")

['εἰμί', 'ἐάω', 'ἵημι', 'ἑός']

In [0]:
### testing on a random subselection of strings
string_lemmatized = []
for number in random.choices(range(0, 1662), k=20):
  string = data_merged_raw.iloc[number]["string"]
  string_lemmatized += lemmatize_string(string, all_lemmata=True, filter_by_postag=None)
nltk.FreqDist(string_lemmatized).most_common(100)

[('λέγω', 4796),
 ('φημί', 4691),
 ('γίγνομαι', 3590),
 ('αὐτοῦ/αὐτός', 2826),
 ('λόγος', 2416),
 ('πᾶς', 2326),
 ('ἔχω', 2085),
 ('ποιέω', 1992),
 ('ἀλλ', 1970),
 ('εἶπον', 1927),
 ('τίς/τις', 1778),
 ('θεός', 1561),
 ('πρότερος', 1545),
 ('ἕτερος', 1514),
 ('ἄνθρωπος', 1476),
 ('λαμβάνω', 1428),
 ('θεάομαι/θεός', 1364),
 ('εἰμί/ἐάν/ἠμί/ἤν', 1351),
 ('ὅς/ὁ', 1290),
 ('ἁμοῦ/ἐμέω/ἐμός/ἁμός/ἡμός/ἐγώ', 1256),
 ('δείκνυμι', 1208),
 ('χριστός', 1202),
 ('ἔθω/καθά/καθό/κατά/εἴθε/εἶμι/μαράν/εἶτα/ἔτης/ἀτάομαι/ἄτη/ἐθάς/ἔτι/εἰμί/ἐτάζω/ἄατος/ἀίτας/εἴτε',
  1178),
 ('οἷος', 1175),
 ('πάλιν', 1141),
 ('γῆ', 1136),
 ('ὑπάρχω', 1115),
 ('ὥσπερ', 1093),
 ('ἐκεῖνος', 1049),
 ('συμβαίνω', 1046),
 ('ἤγουν', 1028),
 ('ἐπ', 1001),
 ('πολύς', 997),
 ('πρότασις', 997),
 ('δεῖ/δέω', 966),
 ('μέγας', 963),
 ('αὐτοὺς', 963),
 ('μᾶλλον/μαλλός', 941),
 ('ἀπόδειξις', 922),
 ('σεύω/σοῦ/σός', 911),
 ('λοιπός', 888),
 ('προφήτης', 875),
 ('ἐπειδη', 859),
 ('κύριος/κῦρος', 858),
 ('δίδωμι', 838),
 ('δοκέω', 835),
 ('μ

In [0]:
 ### testing for a random subselection of documents, print a random sentence
 for number in random.choices(range(0, 1662), k=30):
  string = data_merged_raw.iloc[number]["string"]
  sentences = get_lemmatized_sentences(string)
  print(sentences[random.choice(range(len(sentences)))])

['μέχρι', 'ὕδωρ', 'γράφω', 'κατακύπτω', 'πέτρα', 'ἵστημι', 'γράφω', 'εὐσχήμων', 'στέρνον', 'δείκνυμι', 'ἐπιπόλαιος', 'μίμησις', 'ὕδωρ', 'συνιζάνω', 'μῆκος']
['πάρειμι', 'βιόω', 'διʼ', 'ἔσοπτρος', 'αἰνίγμαει', 'πρόσωπον', 'πρόσωπον', 'ἀλήθεια', 'θεωρέω']
['θαυμάζω', 'μᾶλλον', 'ἐρῶ', 'ἐξαίφνης', 'ἀποδείκνυμι', 'φύω', 'δαιμονάω', 'ἐπιτηδεύω', 'ἀρετή', 'κἂν', '<εἰ>', 'λανθάνω', 'βραχὺ', 'λαμβάνω', 'πάλιν', 'ἀνα']
['περιειλήϲαντεϲ', 'διμερής', 'ϲειράν', 'ἄγω', 'περιϲϲοῦ', 'ἐπείληϲιν', 'ἐγκυκλέομαι', 'θώρακοϲ', 'προϲκαταλαμβανόμενοι', 'τοὺϲ', 'βραχίοναϲ', 'παρατεταγμένουϲ', 'ταῖϲ', 'πλευραῖϲ', 'ἵνα', 'άρμόϲῃ', 'κλείς', 'καταγειϲῶν']
['πλέω', 'οἷος', 'χροιά', 'ἀηρ', 'ὕδωρ', 'ἄμφω', 'διαφανής', 'ἕτερος', 'ἔχω', 'μόνος', 'αἰσθάνομαι', 'ἄμφω']
['ἕτερος', 'ἀπόεἶδον', 'τοιγάρ', 'ἀμείψομαί', 'φημί', 'δικαιοσύνη', 'καλός', 'δίκη', 'δικάζω', 'ἔρχομαι', 'ἦδος', 'ἐπιβαίνω', 'ὄχημα', 'δείξασά', 'ὄχημα', 'ὑπόπτερος', 'ἵππος', 'πήγασος', 'ἔοικα', 'ὅπως', 'οἶδα', 'οἷος', 'ἡλίκος', 'ἀκολουθέω', 'ἀγνοέω', 'μ

CPU times: user 4min 30s, sys: 1.69 s, total: 4min 32s
Wall time: 4min 32s


In [0]:
data_merged_raw.head(5)

,filename,author,title,string,lem_senteces
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...,"[[ανωνυμου, αναμετρησις, της, οικουμενης, ηας,..."
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...,"[[‘], [’], [‘], [‘], [‘], [‘], [’, ‘], [‘], [’..."
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...","[[ἣν], [ἀντιπαρακαλέω], [δι᾿, ἐπιστολή, πινυτό..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...,"[[περι, κοσμου, προς, αλεξανδρον], [πολλακις, ..."
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...,"[[περι, πνευματος], [ἔμφυτος, πνεῦμα, διαμονή,..."


In [0]:
%%time
data_merged_raw["lemmatized"] = data_merged_raw.apply(lambda row: lemmatize_string(row["string"]), axis=1)

CPU times: user 4min 15s, sys: 1.34 s, total: 4min 16s
Wall time: 4min 16s


In [0]:
data_merged_raw.head(5)

,filename,author,title,string,lem_senteces,lemmatized
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...,"[[ανωνυμου, αναμετρησις, της, οικουμενης, ηας,...","[ανωνυμου, αναμετρησις, της, οικουμενης, ηας, ..."
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...,"[[‘], [’], [‘], [‘], [‘], [‘], [’, ‘], [‘], [’...","[ο, ο, ’, ?, ﬁ, ’, ‘, ’, . . . ., ‘, .’, . . ...."
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...","[[ἣν], [ἀντιπαρακαλέω], [δι᾿, ἐπιστολή, πινυτό...","[. . . . ., ἣν, æ, πινυτός, ἀντιγράφω, θαυμάζω..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...,"[[περι, κοσμου, προς, αλεξανδρον], [πολλακις, ...","[περι, κοσμου, προς, αλεξανδρον., πολλακις, θε..."
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...,"[[περι, πνευματος], [ἔμφυτος, πνεῦμα, διαμονή,...","[περι, πνευματος., ἔμφυτος, πνεῦμα, διαμονή, α..."


# Inspecting Aristotle

In [0]:


len(data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")])

39

In [0]:
### inspecting Aristotle from a closer distance
data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]

,filename,author,title,string,lem_senteces,lemmatized
156,tlg0086.tlg001.1st1K-grc1.xml,Aristotle,Analytica priora; Aristotelis Opera,ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. ΠΡΩΤΟΝ εἰπεῖν περὶ τί κ...,"[[αναλυτικων, προτερων, α], [πρωτον, εἶπον, σκ...","[αναλυτικων, προτερων, α., πρωτον, εἶπον, σκέψ..."
157,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora; Ar...,ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. Πρῶτον εἰπεῖν περὶ τί κ...,"[[αναλυτικων, προτερων, α], [πρότερος, εἶπον, ...","[αναλυτικων, προτερων, α., πρότερος, εἶπον, σκ..."
158,tlg0086.tlg002.1st1K-grc1.xml,Aristotle,De anima; Aristotelis Opera,ΠΕΡΙ ΨΥΧΗΣ Α. ΤΩΝ καλῶν καὶ τιμίων τὴν εἴδησιν...,"[[περι, ψυχης, α], [των, καλέω, τιμάω, εἴδομαι...","[περι, ψυχης, α., των, καλέω, τιμάω, εἴδομαι, ..."
159,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima; Aritoteles De anima,ΠEP1 ΨΥΧΗΣ Α Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ...,"[[π], [δοκέω, ἀλήθεια, ἅπας, γνῶσις, μέγας, συ...","[π, ψυχης, α, καλέω, τιμάω, εἴδομαι, ὑπολαμβάν..."
160,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo; Aristoteles De coelo et De generatio...,ΠEPI ΟΥΡΑΝΟΥ A. Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ...,"[[π], [φύσις, ἐπιστήμη, σχεδόν, πλεῖστος, φαίν...","[π, ουρανου, φύσις, ἐπιστήμη, σχεδόν, πλεῖστος..."
161,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae; Aristotelis Opera,ΚΑΤΗΓΟΡΙΑΙ. ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοι...,"[[κατηγοριαι], [ὁμωνυμα, λέγω, ὄνομα, μόνος, κ...","[κατηγοριαι., ὁμωνυμα, λέγω, ὄνομα, μόνος, κοι..."
162,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum; Aristotelis Opera,ΠΕΡΙ ΕΝΥΠΝΙΩΝ ΚΑΙ ΤΗΣ ΚΑΘ’ ΥΠΝΟΝ ΜΑΝΤΙΚΗΣ. Περ...,"[[περι, ενυπνιων, και, της, υπνον, μαντικης], ...","[περι, ενυπνιων, και, της, υπνον, μαντικης., μ..."
163,tlg0086.tlg013.1st1K-grc1.xml,Aristotle,De generatione et corruptione; Aristotle. On S...,ΑΙΣΤΟΤΕΛΟΥΣ ΠΕΡΠ ΓΕΝΕΣΕΩΣ ΚΑΙ ΦΘΟΡΑΣ Α 314 a 1...,"[[αιστοτελους, περπ, γενεσεως, και, φθορας, α]...","[αιστοτελους, περπ, γενεσεως, και, φθορας, α, ..."
164,tlg0086.tlg013.1st1K-grc2.xml,Aristotle,De generatione et corruptione; Aristoteles De ...,ΠΕΡΙ ΙΕΝΕΣΕΩΣ ΚΑΙ ΦΘΟΡΑΣ Α. Περὶ δὲ γενέσεως κ...,"[[περι, ιενεσεως, και, φθορας, α], [γένεσις, φ...","[περι, ιενεσεως, και, φθορας, α., γένεσις, φθο..."
165,tlg0086.tlg014.1st1K-grc1.xml,Aristotle,Historia animalium; Aristotelis Opera,ΠΕΡΙ ΤΑ ΖΩΙΑ ΙΣΤΟΡΙΩΝ Α. ΤΩΝ ἐν τοῖς ζῴοις μορ...,"[[περι, τα, ζωια, ιστοριων, α], [των, ζώιον, μ...","[περι, τα, ζωια, ιστοριων, α., των, ζώιον, μορ..."


In [0]:
arist = data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.tlg010")]["lem_senteces"].tolist()[0]
print(arist)

[['βοοκ', 'πᾶς', 'τέχνη', 'καί', 'πᾶς', 'μέθοδος', 'ὁμοιόω', 'δέ', 'πρᾶξίς', 'καί', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω'], ['διό', 'καλός', 'ἀποφαίνω', 'ἀγαθός', 'ἵημι', 'πάντʼ', 'ἐφίημι'], ['διαφορά', 'δέ', 'φαίνω', 'τελέω'], ['μέν', 'γάρ', 'εἰμί', 'ἐνέργεια', 'δέ', 'παρʼ', 'αὐτός', 'ἔργον'], ['δʼ', 'εἰμί', 'τελέω', 'πηρός', 'πράσσω', 'βελτιόω', 'φύω', 'ἐνέργεια', 'ἔργον'], ['πολύς', 'δέ', 'πρᾶξις', 'εἰμί', 'καί', 'τεχνάζω', 'καί', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'καί', 'τελέω'], ['ἰατρικός', 'μέν', 'γάρ', 'ὑγίεια', 'ναυπηγικός', 'δέ', 'πλοῖον', 'στρατηγικός', 'δέ', 'νικάω', 'οἰκονομικός', 'δέ', 'πλοῦτος'], ['ὅσος', 'δʼ', 'εἰμί', 'τοιοῦτος', 'ὑπό', 'εἷς', 'δύναμις', 'καθά', 'ὑπό', 'την', 'ἱππικην', 'χαλινοποιικη', 'καί', 'ὅσος', 'ἱππικός', 'ὀργανόω', 'εἰμί', 'δέ', 'καί', 'πᾶς', 'πολεμικη', 'πρᾶξις', 'ὑπό', 'την', 'στρατηγικός', 'κατά', 'αὐτός', 'δη', 'τρόπος', 'ὑφʼ', 'ἕτερος'], ['ἅπας', 'δέ', 'ἀρχιτεκτονικός', 'τελέω', 'πᾶς', 'εἰμί', 'αἱρετός', 'ὑπʼ', 'αὐτός'], ['γάρ

In [0]:
data_merged_raw.to_json("temp.json")

In [0]:
s.put(sddk_url + "SDAM_data/OGL/data_lemmatized.json",  data=data_merged_raw.to_json())

OverflowError: ignored